In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/notebook368ff20ef1/__results__.html
/kaggle/input/notebook368ff20ef1/study.pkl
/kaggle/input/notebook368ff20ef1/__notebook__.ipynb
/kaggle/input/notebook368ff20ef1/__output__.json
/kaggle/input/notebook368ff20ef1/custom.css
/kaggle/input/m5-forecasting-accuracy/calendar.csv
/kaggle/input/m5-forecasting-accuracy/sample_submission.csv
/kaggle/input/m5-forecasting-accuracy/sell_prices.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_validation.csv
/kaggle/input/m5-forecasting-accuracy/sales_train_evaluation.csv


In [3]:
from sklearn.metrics import mean_squared_error
import lightgbm as lgb
import lightgbm
from lightgbm import LGBMRegressor
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import pickle
import gc
import optuna
from optuna.integration import LightGBMPruningCallback
import time

In [6]:
data=pickle.load(open('/kaggle/input/m5-model-features/processed_pickle.pkl','rb'))

In [7]:
cat_column=[]
for i in data.columns:
    if(str(data.dtypes[i])=='category'):
        cat_column.append(i)

In [8]:
cat_column
gc.collect()


91

In [9]:
data=data[data['d']>=1970-(365*3)+(3*28)]

In [23]:
def objective(trial,data):
    param_grid = {
        "n_estimators": trial.suggest_int("n_estimators", 100,600),
        'subsample':trial.suggest_float("subsample", 0.1,1.0 ),
        'feature_fraction':trial.suggest_float("feature_fraction", 0.1,1.0 ),
        'num_iterations ':trial.suggest_int("num_iterations", 500,1500,20 ),
        'max_bin ':trial.suggest_int("max_bin", 100,500,20 ),
        "learning_rate": trial.suggest_float("learning_rate", 0.001,0.5 ),
        "lambda_l2": trial.suggest_float("lambda_l2", 0.001,0.5 ),
        "num_leaves": trial.suggest_int("num_leaves", 20, 200, step=20),
        "max_depth": trial.suggest_int("max_depth", 100, 500,step=20),
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 50,200, step=10),
        #'verbose':trial.suggest_categorical('verbose',[-1]),
    }

    cv_scores = np.empty(3)

    for i in range(3):
        
        start_time = time.time()
        
        model=LGBMRegressor(param_grid,force_row_wise= True,n_jobs=-1)
        
        model.__init__(boosting_type='gbdt', metric='rmse',force_row_wise= True)
        j=1829
        k=1829+28

        x_train=data[(data['d']>=(j-(365*3)+(i*28)))&(data['d']<j+(i*28))].drop('sold',axis=1)
        y_train=data[(data['d']>=(j-(365*3)+(i*28)))&(data['d']<j+(i*28))]['sold']
        
        x_cv=data[(data['d']>=j+(i*28))&(data['d']<k+(i*28))].drop('sold',axis=1)
        y_cv=data[(data['d']>=j+(i*28))&(data['d']<k+(i*28))]['sold']
        
        model.fit(X=x_train,y=y_train,
                  eval_set=[(x_cv,y_cv)],
                  eval_metric=['rmse'],callbacks=[LightGBMPruningCallback(trial, "rmse")]
                  ,verbose=100,categorical_feature=cat_column,early_stopping_rounds=20)
        # 
        preds = model.predict(x_cv)
        train_pred=model.predict(x_train)
        
        res=np.sqrt(mean_squared_error(preds, y_cv))
        train_res=np.sqrt(mean_squared_error(train_pred, y_train))
        
        print('Test RMSE',res)
        print('Train RMSE', train_res)
        idx=0
        cv_scores[idx] = res
        idx+=1
        print('Time taken for one iteration', (time.time()-start_time)/60)
        
        gc.collect()
    return np.mean(cv_scores)

In [25]:
gc.collect()
study = optuna.create_study(direction="minimize", study_name="LGBM_Regression_1")
func = lambda trial: objective(trial, data)
study.optimize(func, n_trials=25)

[I 2023-03-02 17:51:47,780] A new study created in memory with name: LGBM_Regression_1


[100]	valid_0's rmse: 2.37592
Test RMSE 2.3732546127145
Train RMSE 2.370410848929493
Time taken for one iteration 4.582739885648092
[100]	valid_0's rmse: 2.27782
Test RMSE 2.277824866035757
Train RMSE 2.350132563306263
Time taken for one iteration 4.757102004686991
[100]	valid_0's rmse: 2.19393


[I 2023-03-02 18:06:07,145] Trial 0 finished with value: 0.7313084000708882 and parameters: {'n_estimators': 435, 'subsample': 0.8938259768268002, 'feature_fraction': 0.3552811134343441, 'num_iterations': 1240, 'max_bin': 240, 'learning_rate': 0.4010889371929507, 'lambda_l2': 0.10388429583901222, 'num_leaves': 20, 'max_depth': 200, 'min_data_in_leaf': 160}. Best is trial 0 with value: 0.7313084000708882.


Test RMSE 2.1939252002126644
Train RMSE 2.3506333065475595
Time taken for one iteration 4.974971675872803
[100]	valid_0's rmse: 2.37592
Test RMSE 2.3732546127145
Train RMSE 2.370410848929493
Time taken for one iteration 4.5341124375661215
[100]	valid_0's rmse: 2.27782
Test RMSE 2.277824866035757
Train RMSE 2.350132563306263
Time taken for one iteration 4.771618048350017
[100]	valid_0's rmse: 2.19393


[I 2023-03-02 18:20:26,867] Trial 1 finished with value: 0.7313084000708882 and parameters: {'n_estimators': 337, 'subsample': 0.8543944175049726, 'feature_fraction': 0.8974434991754454, 'num_iterations': 960, 'max_bin': 500, 'learning_rate': 0.2677282448161616, 'lambda_l2': 0.3925209486869428, 'num_leaves': 120, 'max_depth': 120, 'min_data_in_leaf': 190}. Best is trial 0 with value: 0.7313084000708882.


...

[I 2023-03-02 19:03:50,657] Trial 4 finished with value: 0.7313084000708882 and parameters: {'n_estimators': 102, 'subsample': 0.16662624762445957, 'feature_fraction': 0.705073524098159, 'num_iterations': 1420, 'max_bin': 160, 'learning_rate': 0.192779399100321, 'lambda_l2': 0.16204497756637004, 'num_leaves': 40, 'max_depth': 140, 'min_data_in_leaf': 70}. Best is trial 0 with value: 0.7313084000708882.


Test RMSE 2.1939252002126644
Train RMSE 2.3506333065475595
Time taken for one iteration 4.9276758154233296


[I 2023-03-02 19:05:47,114] Trial 5 pruned. Trial was pruned at iteration 29.
[I 2023-03-02 19:07:20,130] Trial 6 pruned. Trial was pruned at iteration 9.
[I 2023-03-02 19:09:25,132] Trial 7 pruned. Trial was pruned at iteration 36.
[I 2023-03-02 19:10:32,302] Trial 8 pruned. Trial was pruned at iteration 3.
[I 2023-03-02 19:11:35,033] Trial 9 pruned. Trial was pruned at iteration 3.
[I 2023-03-02 19:12:42,433] Trial 10 pruned. Trial was pruned at iteration 3.
[I 2023-03-02 19:14:39,107] Trial 11 pruned. Trial was pruned at iteration 29.
[I 2023-03-02 19:16:32,219] Trial 12 pruned. Trial was pruned at iteration 29.
[I 2023-03-02 19:17:35,320] Trial 13 pruned. Trial was pruned at iteration 3.
[I 2023-03-02 19:18:41,811] Trial 14 pruned. Trial was pruned at iteration 3.
[I 2023-03-02 19:20:34,334] Trial 15 pruned. Trial was pruned at iteration 29.
[I 2023-03-02 19:22:30,845] Trial 16 pruned. Trial was pruned at iteration 29.
[I 2023-03-02 19:24:37,107] Trial 17 pruned. Trial was pruned a

[100]	valid_0's rmse: 2.37592
Test RMSE 2.3732546127145
Train RMSE 2.370410848929493
Time taken for one iteration 4.4750101606051125
[100]	valid_0's rmse: 2.27782
Test RMSE 2.277824866035757
Train RMSE 2.350132563306263
Time taken for one iteration 4.846396092573801
[100]	valid_0's rmse: 2.19393
Test RMSE 2.1939252002126644
Train RMSE 2.3506333065475595
Time taken for one iteration 4.904734774430593


[I 2023-03-02 19:47:24,782] Trial 23 finished with value: 0.47602511837220396 and parameters: {'n_estimators': 223, 'subsample': 0.8213602838283158, 'feature_fraction': 0.31236821047432106, 'num_iterations': 740, 'max_bin': 220, 'learning_rate': 0.10635904537034443, 'lambda_l2': 0.12178091743096875, 'num_leaves': 120, 'max_depth': 360, 'min_data_in_leaf': 200}. Best is trial 23 with value: 0.47602511837220396.
[I 2023-03-02 19:49:16,243] Trial 24 pruned. Trial was pruned at iteration 29.


In [26]:
'''import pickle
pickle.dump(study,open('study.pkl','wb'))'''

In [4]:
study=pickle.load(open('/kaggle/input/notebook368ff20ef1/study.pkl','rb'))

In [6]:
study.trials_dataframe()

,number,value,datetime_start,datetime_complete,duration,params_feature_fraction,params_lambda_l2,params_learning_rate,params_max_bin,params_max_depth,params_min_data_in_leaf,params_n_estimators,params_num_iterations,params_num_leaves,params_subsample,state
0,0,0.731308,2023-03-02 17:54:29.889649,2023-03-02 18:09:05.947966,0 days 00:14:36.058317,0.949606,0.247752,0.454116,480,340,130,567,600,100,0.248774,COMPLETE
1,1,0.731308,2023-03-02 18:09:05.950532,2023-03-02 18:23:53.837727,0 days 00:14:47.887195,0.234016,0.306786,0.420884,180,420,150,300,1100,40,0.317236,COMPLETE
2,2,0.731308,2023-03-02 18:23:53.840457,2023-03-02 18:38:24.260108,0 days 00:14:30.419651,0.447029,0.351205,0.215870,380,100,190,258,800,160,0.279919,COMPLETE
3,3,0.731308,2023-03-02 18:38:24.262572,2023-03-02 18:53:08.389054,0 days 00:14:44.126482,0.567272,0.368710,0.387511,440,260,100,142,1140,80,0.343110,COMPLETE
4,4,0.731308,2023-03-02 18:53:08.391626,2023-03-02 19:07:48.617612,0 days 00:14:40.225986,0.123107,0.084690,0.470423,340,300,60,381,920,140,0.632529,COMPLETE
5,5,2.405469,2023-03-02 19:07:48.619934,2023-03-02 19:09:57.387655,0 days 00:02:08.767721,0.437793,0.280579,0.004067,460,500,100,481,740,80,0.846477,PRUNED
6,6,2.421172,2023-03-02 19:09:57.389856,2023-03-02 19:12:05.752033,0 days 00:02:08.362177,0.191016,0.488840,0.361801,180,380,90,293,1360,40,0.472633,PRUNED
7,7,2.421172,2023-03-02 19:12:05.754110,2023-03-02 19:14:05.087210,0 days 00:01:59.333100,0.267562,0.320817,0.475701,500,380,190,383,1380,180,0.661909,PRUNED
8,8,2.539224,2023-03-02 19:14:05.089403,2023-03-02 19:15:32.457856,0 days 00:01:27.368453,0.159814,0.205062,0.383671,440,460,50,507,840,40,0.728198,PRUNED
9,9,2.539224,2023-03-02 19:15:32.459828,2023-03-02 19:16:59.342365,0 days 00:01:26.882537,0.695227,0.413900,0.253930,360,280,120,289,580,200,0.383749,PRUNED


In [6]:
print(f"\tBest value (rmse): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (rmse): 0.73131
	Best params:
		n_estimators: 567
		subsample: 0.24877363747333814
		feature_fraction: 0.9496060936873285
		num_iterations: 600
		max_bin: 480
		learning_rate: 0.4541157216562791
		lambda_l2: 0.24775163612644088
		num_leaves: 100
		max_depth: 340
		min_data_in_leaf: 130
